In [2]:
from lxml import html
from lxml.html import fromstring
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import datetime

In [3]:
games = pd.read_csv("...\\spreadspoke_scores.csv")
acs = pd.read_csv("...\\acron.csv")

In [4]:
games

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,winner
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71,NaN,away
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70,NaN,home
2,9/4/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82,NaN,home
3,9/9/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78,NaN,away
4,9/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62,NaN,home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12929,1/3/2021,2020,17,False,New England Patriots,NaN,NaN,New York Jets,NaN,NaN,NaN,Gillette Stadium,False,NaN,NaN,NaN,NaN,NaN
12930,1/3/2021,2020,17,False,New York Giants,NaN,NaN,Dallas Cowboys,NaN,NaN,NaN,MetLife Stadium,False,NaN,NaN,NaN,NaN,NaN
12931,1/3/2021,2020,17,False,Philadelphia Eagles,NaN,NaN,Washington Football Team,NaN,NaN,NaN,Lincoln Financial Field,False,NaN,NaN,NaN,NaN,NaN
12932,1/3/2021,2020,17,False,San Francisco 49ers,NaN,NaN,Seattle Seahawks,NaN,NaN,NaN,Levi's Stadium,False,NaN,NaN,NaN,NaN,NaN


In [5]:
acs

,team,acronym
0,Miami Dolphins,mia
1,Houston Oilers,oti
2,San Diego Chargers,sdg
3,Green Bay Packers,gnb
4,Atlanta Falcons,atl
5,Buffalo Bills,buf
6,Detroit Lions,det
7,Pittsburgh Steelers,pit
8,San Francisco 49ers,sfo
9,St. Louis Cardinals,crd


In [ ]:
dates = games[['schedule_date', 'team_home']]
for p , stuff in dates.iterrows():
    try:
        print(stuff[0])
        team = stuff[1]
        datestuff = stuff[0]
        print(datestuff.split('/')[0])
        print(datestuff.split('/')[1])
        print(datestuff.split('/')[2])
        print(team)
        ac = acs.loc[acs['team'] == team]
        #print(ac['acronym'])
        l = ac['acronym']
        for item in l:
            acro = item
        print(acro)
    except:
        print("failed")

In [10]:
header = ['Date', 'Player', 'Tm', 'p_Cmp', 'p_Att', 'p_Yds', 'p_TD', 'Int', 'Sk', 'Yds_lost', 'p_Lng', 'p_Rate', 'ru_Att', 'ru_Yds', 'ru_TD', 'ru_Lng', 're_targ','Rec', 're_Yds', 're_TD', 're_Lng', 'fmb', 'fmb_loss']
offdf = pd.DataFrame(columns = header)

test = dates.head()

print(datetime.datetime.now())
for pad, item in dates.iterrows():
    try:
        team = item[1]
        day = item[0].split('/')
        if len(day[0]) == 1:
            sday = f'0{day[0]}'
        else:
            sday = day[0]
        if len(day[1]) == 1:
            mday = f'0{day[1]}'
        else:
            mday = day[1]  
        ac = acs.loc[acs['team'] == team]
        l = ac['acronym']
        for fe in l:
            acro = fe
        url = f"https://www.pro-football-reference.com/boxscores/{day[2]}{sday}{mday}0{acro}.htm"
        print(url)
        response = requests.get(url)
        root = fromstring(response.content)
        table = root.xpath('//table[@id="player_offense"]')[0]
        for row in table.xpath(".//tr")[1:]:
            if [cell.text_content() for cell in row.xpath(".//th")] == header:
                print("")
            elif len([cell.text_content() for cell in row.xpath(".//th")]) == 1:
                player = [cell.text_content() for cell in row.xpath(".//th")]
                scores = [cell.text_content() for cell in row.xpath(".//td")]
                points = scores[1:]
                if int(day[2]) < 1992:
                    offdf = offdf.append({'Date': item[0], 'Player': player[0], 'Tm': scores[0] , 'p_Cmp': points[0], 'p_Att': points[1], 'p_Yds': points[2], 'p_TD': points[3], 'Int': points[4], 'Sk': points[5], 'Yds_lost': points[6], 'p_Lng': points[7], 'p_Rate': points[8], 'ru_Att': points[9], 'ru_Yds': points[10], 'ru_TD': points[11], 'ru_Lng': points[12], 'Rec': points[13], 're_Yds': points[14], 're_TD': points[15], 're_Lng': points[16]},ignore_index=True)
                elif int(day[2]) >= 1992 and int(day[2]) < 1994:
                    offdf = offdf.append({'Date': item[0], 'Player': player[0], 'Tm': scores[0] , 'p_Cmp': points[0], 'p_Att': points[1], 'p_Yds': points[2], 'p_TD': points[3], 'Int': points[4], 'Sk': points[5], 'Yds_lost': points[6], 'p_Lng': points[7], 'p_Rate': points[8], 'ru_Att': points[9], 'ru_Yds': points[10], 'ru_TD': points[11], 'ru_Lng': points[12],'re_targ': points[13], 'Rec': points[14], 're_Yds': points[15], 're_TD': points[16], 're_Lng': points[17]},ignore_index=True)
                else:
                    offdf = offdf.append({'Date': item[0], 'Player': player[0], 'Tm': scores[0] , 'p_Cmp': points[0], 'p_Att': points[1], 'p_Yds': points[2], 'p_TD': points[3], 'Int': points[4], 'Sk': points[5], 'Yds_lost': points[6], 'p_Lng': points[7], 'p_Rate': points[8], 'ru_Att': points[9], 'ru_Yds': points[10], 'ru_TD': points[11], 'ru_Lng': points[12],'re_targ': points[13], 'Rec': points[14], 're_Yds': points[15], 're_TD': points[16], 're_Lng': points[17], 'fmb': points[18], 'fmb_loss': points[19]},ignore_index=True)
    except:
        print("Last Site Failed")

print(datetime.datetime.now())

2020-11-14 14:10:03.328777
https://www.pro-football-reference.com/boxscores/196609020mia.htm
https://www.pro-football-reference.com/boxscores/196609030oti.htm
https://www.pro-football-reference.com/boxscores/196609040sdg.htm
https://www.pro-football-reference.com/boxscores/196609090mia.htm
https://www.pro-football-reference.com/boxscores/196609100gnb.htm
https://www.pro-football-reference.com/boxscores/196609100oti.htm
https://www.pro-football-reference.com/boxscores/196609100sdg.htm
https://www.pro-football-reference.com/boxscores/196609110atl.htm
https://www.pro-football-reference.com/boxscores/196609110buf.htm
https://www.pro-football-reference.com/boxscores/196609110det.htm
https://www.pro-football-reference.com/boxscores/196609110pit.htm
https://www.pro-football-reference.com/boxscores/196609110sfo.htm
https://www.pro-football-reference.com/boxscores/196609110crd.htm
https://www.pro-football-reference.com/boxscores/196609110was.htm
https://www.pro-football-reference.com/boxscores/

https://www.pro-football-reference.com/boxscores/196611240det.htm
https://www.pro-football-reference.com/boxscores/196611240rai.htm
https://www.pro-football-reference.com/boxscores/196611270clt.htm
https://www.pro-football-reference.com/boxscores/196611270chi.htm
https://www.pro-football-reference.com/boxscores/196611270den.htm
https://www.pro-football-reference.com/boxscores/196611270mia.htm
https://www.pro-football-reference.com/boxscores/196611270min.htm
https://www.pro-football-reference.com/boxscores/196611270nyj.htm
https://www.pro-football-reference.com/boxscores/196611270crd.htm
https://www.pro-football-reference.com/boxscores/196611270was.htm
https://www.pro-football-reference.com/boxscores/196612030rai.htm
https://www.pro-football-reference.com/boxscores/196612040clt.htm
https://www.pro-football-reference.com/boxscores/196612040nwe.htm
https://www.pro-football-reference.com/boxscores/196612040cle.htm
https://www.pro-football-reference.com/boxscores/196612040dal.htm
https://ww

https://www.pro-football-reference.com/boxscores/196710290kan.htm
https://www.pro-football-reference.com/boxscores/196710290nor.htm
https://www.pro-football-reference.com/boxscores/196710290nyg.htm
https://www.pro-football-reference.com/boxscores/196710290nyj.htm
https://www.pro-football-reference.com/boxscores/196710290rai.htm
https://www.pro-football-reference.com/boxscores/196710290phi.htm
https://www.pro-football-reference.com/boxscores/196710290sfo.htm
https://www.pro-football-reference.com/boxscores/196710290was.htm
https://www.pro-football-reference.com/boxscores/196710300crd.htm
https://www.pro-football-reference.com/boxscores/196711050clt.htm
https://www.pro-football-reference.com/boxscores/196711050nwe.htm
https://www.pro-football-reference.com/boxscores/196711050buf.htm
https://www.pro-football-reference.com/boxscores/196711050dal.htm
https://www.pro-football-reference.com/boxscores/196711050den.htm
https://www.pro-football-reference.com/boxscores/196711050det.htm
https://ww

https://www.pro-football-reference.com/boxscores/196809220ram.htm
https://www.pro-football-reference.com/boxscores/196809220nor.htm
https://www.pro-football-reference.com/boxscores/196809220phi.htm
https://www.pro-football-reference.com/boxscores/196809220sfo.htm
https://www.pro-football-reference.com/boxscores/196809280mia.htm
https://www.pro-football-reference.com/boxscores/196809290buf.htm
https://www.pro-football-reference.com/boxscores/196809290cin.htm
https://www.pro-football-reference.com/boxscores/196809290cle.htm
https://www.pro-football-reference.com/boxscores/196809290den.htm
https://www.pro-football-reference.com/boxscores/196809290gnb.htm
https://www.pro-football-reference.com/boxscores/196809290oti.htm
https://www.pro-football-reference.com/boxscores/196809290min.htm
https://www.pro-football-reference.com/boxscores/196809290nor.htm
https://www.pro-football-reference.com/boxscores/196809290nyg.htm
https://www.pro-football-reference.com/boxscores/196809290phi.htm
https://ww

https://www.pro-football-reference.com/boxscores/196812010nwe.htm
https://www.pro-football-reference.com/boxscores/196812010cle.htm
https://www.pro-football-reference.com/boxscores/196812010den.htm
https://www.pro-football-reference.com/boxscores/196812010min.htm
https://www.pro-football-reference.com/boxscores/196812010nor.htm
https://www.pro-football-reference.com/boxscores/196812010nyj.htm
https://www.pro-football-reference.com/boxscores/196812010pit.htm
https://www.pro-football-reference.com/boxscores/196812010sfo.htm
https://www.pro-football-reference.com/boxscores/196812070gnb.htm
https://www.pro-football-reference.com/boxscores/196812070oti.htm
https://www.pro-football-reference.com/boxscores/196812080atl.htm
https://www.pro-football-reference.com/boxscores/196812080dal.htm
https://www.pro-football-reference.com/boxscores/196812080ram.htm
https://www.pro-football-reference.com/boxscores/196812080mia.htm
https://www.pro-football-reference.com/boxscores/196812080nyg.htm
https://ww

...

https://www.pro-football-reference.com/boxscores/201909220tam.htm
https://www.pro-football-reference.com/boxscores/201909230was.htm
https://www.pro-football-reference.com/boxscores/201909260gnb.htm
https://www.pro-football-reference.com/boxscores/201909290crd.htm
https://www.pro-football-reference.com/boxscores/201909290atl.htm
https://www.pro-football-reference.com/boxscores/201909290rav.htm
https://www.pro-football-reference.com/boxscores/201909290buf.htm
https://www.pro-football-reference.com/boxscores/201909290chi.htm
https://www.pro-football-reference.com/boxscores/201909290den.htm
https://www.pro-football-reference.com/boxscores/201909290det.htm
https://www.pro-football-reference.com/boxscores/201909290htx.htm
https://www.pro-football-reference.com/boxscores/201909290clt.htm
https://www.pro-football-reference.com/boxscores/201909290ram.htm
https://www.pro-football-reference.com/boxscores/201909290mia.htm
https://www.pro-football-reference.com/boxscores/201909290nor.htm
https://ww

https://www.pro-football-reference.com/boxscores/201911240phi.htm
https://www.pro-football-reference.com/boxscores/201911240sfo.htm
https://www.pro-football-reference.com/boxscores/201911240oti.htm
https://www.pro-football-reference.com/boxscores/201911240was.htm
https://www.pro-football-reference.com/boxscores/201911250ram.htm
https://www.pro-football-reference.com/boxscores/201911280atl.htm
https://www.pro-football-reference.com/boxscores/201911280dal.htm
https://www.pro-football-reference.com/boxscores/201911280det.htm
https://www.pro-football-reference.com/boxscores/201912010crd.htm
https://www.pro-football-reference.com/boxscores/201912010rav.htm
https://www.pro-football-reference.com/boxscores/201912010car.htm
https://www.pro-football-reference.com/boxscores/201912010cin.htm
https://www.pro-football-reference.com/boxscores/201912010den.htm
https://www.pro-football-reference.com/boxscores/201912010htx.htm
https://www.pro-football-reference.com/boxscores/201912010clt.htm
https://ww

https://www.pro-football-reference.com/boxscores/202009200tam.htm
https://www.pro-football-reference.com/boxscores/202009200oti.htm
https://www.pro-football-reference.com/boxscores/202009210rai.htm
https://www.pro-football-reference.com/boxscores/202009240jax.htm
https://www.pro-football-reference.com/boxscores/202009270crd.htm
https://www.pro-football-reference.com/boxscores/202009270atl.htm
https://www.pro-football-reference.com/boxscores/202009270buf.htm
https://www.pro-football-reference.com/boxscores/202009270cle.htm
https://www.pro-football-reference.com/boxscores/202009270den.htm
https://www.pro-football-reference.com/boxscores/202009270clt.htm
https://www.pro-football-reference.com/boxscores/202009270sdg.htm
https://www.pro-football-reference.com/boxscores/202009270min.htm
https://www.pro-football-reference.com/boxscores/202009270nwe.htm
https://www.pro-football-reference.com/boxscores/202009270nor.htm
https://www.pro-football-reference.com/boxscores/202009270nyg.htm
https://ww

Last Site Failed
https://www.pro-football-reference.com/boxscores/202011220den.htm
Last Site Failed
https://www.pro-football-reference.com/boxscores/202011220htx.htm
Last Site Failed
https://www.pro-football-reference.com/boxscores/202011220clt.htm
Last Site Failed
https://www.pro-football-reference.com/boxscores/202011220jax.htm
Last Site Failed
https://www.pro-football-reference.com/boxscores/202011220rai.htm
Last Site Failed
https://www.pro-football-reference.com/boxscores/202011220sdg.htm
Last Site Failed
https://www.pro-football-reference.com/boxscores/202011220min.htm
Last Site Failed
https://www.pro-football-reference.com/boxscores/202011220nor.htm
Last Site Failed
https://www.pro-football-reference.com/boxscores/202011220was.htm
Last Site Failed
https://www.pro-football-reference.com/boxscores/202011230tam.htm
Last Site Failed
https://www.pro-football-reference.com/boxscores/202011260dal.htm
Last Site Failed
https://www.pro-football-reference.com/boxscores/202011260det.htm
Last

Last Site Failed
https://www.pro-football-reference.com/boxscores/202101030nwe.htm
Last Site Failed
https://www.pro-football-reference.com/boxscores/202101030nyg.htm
Last Site Failed
https://www.pro-football-reference.com/boxscores/202101030phi.htm
Last Site Failed
https://www.pro-football-reference.com/boxscores/202101030sfo.htm
Last Site Failed
https://www.pro-football-reference.com/boxscores/202101030tam.htm
Last Site Failed
2020-11-14 20:15:50.641164


In [11]:
offdf

,Date,Player,Tm,p_Cmp,p_Att,p_Yds,p_TD,Int,Sk,Yds_lost,...,ru_Yds,ru_TD,ru_Lng,re_targ,Rec,re_Yds,re_TD,re_Lng,fmb,fmb_loss
0,9/2/1966,Tom Flores,OAK,11,20,161,2,1,,,...,0,0,0,NaN,0,0,0,0,NaN,NaN
1,9/2/1966,Cotton Davidson,OAK,7,19,80,0,3,,,...,-4,0,0,NaN,0,0,0,0,NaN,NaN
2,9/2/1966,Roger Hagberg,OAK,0,0,0,0,0,,,...,23,0,0,NaN,2,37,0,0,NaN,NaN
3,9/2/1966,Hewritt Dixon,OAK,0,0,0,0,0,,,...,5,1,0,NaN,0,0,0,0,NaN,NaN
4,9/2/1966,Art Powell,OAK,0,0,0,0,0,,,...,0,0,0,NaN,8,104,1,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170208,11/12/2020,A.J. Brown,TEN,0,0,0,0,0,0,0,...,0,0,0,4,1,21,0,21,0,0
170209,11/12/2020,Khari Blasingame,TEN,0,0,0,0,0,0,0,...,0,0,0,1,1,13,0,13,0,0
170210,11/12/2020,Cameron Batson,TEN,0,0,0,0,0,0,0,...,0,0,0,2,2,8,0,5,0,0
170211,11/12/2020,Anthony Firkser,TEN,0,0,0,0,0,0,0,...,0,0,0,3,1,5,0,5,0,0


In [6]:
test = dates.head()
test

,schedule_date,team_home
0,9/2/1966,Miami Dolphins
1,9/3/1966,Houston Oilers
2,9/4/1966,San Diego Chargers
3,9/9/1966,Miami Dolphins
4,9/10/1966,Green Bay Packers


In [12]:
offdf.to_csv('...\\offense2.csv')

In [27]:
a = (22/36-.3)*5
b = (340/36)*.25
c = (3/36)*20
d = 2.375 - (0/36*25)

print(((a+b+c+d)/6)*100)

132.6388888888889


In [11]:
print(datetime.datetime.now())

2020-11-14 14:06:04.737547


In [38]:
header = ['Date', 'Player', 'Tm', 'p_Cmp', 'p_Att', 'p_Yds', 'p_TD', 'Int', 'Sk', 'Yds_lost', 'p_Lng', 'p_Rate', 'ru_Att', 'ru_Yds', 'ru_TD', 'ru_Lng', 're_targ','Rec', 're_Yds', 're_TD', 're_Lng', 'fmb', 'fmb_loss']
offdf = pd.DataFrame(columns = header)

test = dates.head()

print(datetime.datetime.now())
for pad, item in dates.iterrows():
    team = item[1]
    day = item[0].split('/')
    if int(day[2]) >= 1994 and int(day[2]) < 2008:
        if len(day[0]) == 1:
            sday = f'0{day[0]}'
        else:
            sday = day[0]
        if len(day[1]) == 1:
            mday = f'0{day[1]}'
        else:
            mday = day[1]  
        ac = acs.loc[acs['team'] == team]
        l = ac['acronym']
        for fe in l:
            acro = fe
        url = f"https://www.pro-football-reference.com/boxscores/{day[2]}{sday}{mday}0{acro}.htm"
        print(url)
        response = requests.get(url)
        root = fromstring(response.content)
        try:
            table = root.xpath('//table[@id="player_offense"]')[0]
            for row in table.xpath(".//tr")[1:]:
                if [cell.text_content() for cell in row.xpath(".//th")] == header:
                    print("")
                elif len([cell.text_content() for cell in row.xpath(".//th")]) == 1:
                    player = [cell.text_content() for cell in row.xpath(".//th")]
                    scores = [cell.text_content() for cell in row.xpath(".//td")]
                    points = scores[1:]
                    #print(points[19])
                    offdf = offdf.append({'Date': item[0], 'Player': player[0], 'Tm': scores[0] , 'p_Cmp': points[0], 'p_Att': points[1], 'p_Yds': points[2], 'p_TD': points[3], 'Int': points[4], 'Sk': points[5], 'Yds_lost': points[6], 'p_Lng': points[7], 'p_Rate': points[8], 'ru_Att': points[9], 'ru_Yds': points[10], 'ru_TD': points[11], 'ru_Lng': points[12],'re_targ': points[13], 'Rec': points[14], 're_Yds': points[15], 're_TD': points[16], 're_Lng': points[17]},ignore_index=True)
        except:
            print("Last site failed")
                #print(f"'Date': {item[0]}, 'Player': {player[0]}, 'Tm': {scores[0]}, 'p_Cmp': {points[0]}, 'p_Att': {points[1]}, 'p_Yds': {points[2]}, 'p_TD': {points[3]}, 'Int': {points[4]}, 'Sk': {points[5]}, 'Yds_lost': {points[6]}, 'p_Lng': {points[7]}, 'p_Rate': {points[8]}, 'ru_Att': {points[9]}, 'ru_Yds': {points[10]}, 'ru_TD': {points[11]}, 'ru_Lng': {points[12]},'re_targ': {points[13]}, 'Rec': {points[14]}, 're_Yds': {points[15]}, 're_TD': {points[16]}, 're_Lng': {points[17]}, 'fmb': {points[18]}, 'fmb_loss': {points[19]}")
#                 elif:
#                     offdf = offdf.append({'Date': item[0], 'Player': player[0], 'Tm': scores[0], 'p_Cmp': points[0], 'p_Att': points[1], 'p_Yds': points[2], 'p_TD': points[3], 'Int': points[4], 'Sk': points[5], 'Yds_lost': points[6], 'p_Lng': points[7], 'p_Rate': points[8], 'ru_Att': points[9], 'ru_Yds': points[10], 'ru_TD': points[11], 'ru_Lng': points[12],'re_targ': points[13], 'Rec': points[14], 're_Yds': points[15], 're_TD': points[16], 're_Lng': points[17], 'fmb': points[18], 'fmb_loss': points[19]}, ignore_index=True)

2020-11-16 11:14:58.259188
https://www.pro-football-reference.com/boxscores/199401020atl.htm
https://www.pro-football-reference.com/boxscores/199401020det.htm
https://www.pro-football-reference.com/boxscores/199401020oti.htm
https://www.pro-football-reference.com/boxscores/199401020clt.htm
https://www.pro-football-reference.com/boxscores/199401020kan.htm
https://www.pro-football-reference.com/boxscores/199401020rai.htm
https://www.pro-football-reference.com/boxscores/199401020ram.htm
https://www.pro-football-reference.com/boxscores/199401020nwe.htm
https://www.pro-football-reference.com/boxscores/199401020nor.htm
https://www.pro-football-reference.com/boxscores/199401020nyg.htm
https://www.pro-football-reference.com/boxscores/199401020pit.htm
https://www.pro-football-reference.com/boxscores/199401020tam.htm
https://www.pro-football-reference.com/boxscores/199401030sfo.htm
https://www.pro-football-reference.com/boxscores/199401080det.htm
https://www.pro-football-reference.com/boxscores/

https://www.pro-football-reference.com/boxscores/199410240phi.htm
https://www.pro-football-reference.com/boxscores/199410300crd.htm
https://www.pro-football-reference.com/boxscores/199410300buf.htm
https://www.pro-football-reference.com/boxscores/199410300cin.htm
https://www.pro-football-reference.com/boxscores/199410300den.htm
https://www.pro-football-reference.com/boxscores/199410300clt.htm
https://www.pro-football-reference.com/boxscores/199410300rai.htm
https://www.pro-football-reference.com/boxscores/199410300nwe.htm
https://www.pro-football-reference.com/boxscores/199410300nyg.htm
https://www.pro-football-reference.com/boxscores/199410300sdg.htm
https://www.pro-football-reference.com/boxscores/199410300tam.htm
https://www.pro-football-reference.com/boxscores/199410300was.htm
https://www.pro-football-reference.com/boxscores/199410310chi.htm
https://www.pro-football-reference.com/boxscores/199411060atl.htm
https://www.pro-football-reference.com/boxscores/199411060cle.htm
https://ww

https://www.pro-football-reference.com/boxscores/199412310mia.htm
https://www.pro-football-reference.com/boxscores/199501010cle.htm
https://www.pro-football-reference.com/boxscores/199501010min.htm
https://www.pro-football-reference.com/boxscores/199501070pit.htm
https://www.pro-football-reference.com/boxscores/199501070sfo.htm
https://www.pro-football-reference.com/boxscores/199501080dal.htm
https://www.pro-football-reference.com/boxscores/199501080sdg.htm
https://www.pro-football-reference.com/boxscores/199501150pit.htm
https://www.pro-football-reference.com/boxscores/199501150sfo.htm
https://www.pro-football-reference.com/boxscores/199501290sfo.htm
Last site failed
https://www.pro-football-reference.com/boxscores/199509030atl.htm
https://www.pro-football-reference.com/boxscores/199509030chi.htm
https://www.pro-football-reference.com/boxscores/199509030den.htm
https://www.pro-football-reference.com/boxscores/199509030gnb.htm
https://www.pro-football-reference.com/boxscores/199509030c

https://www.pro-football-reference.com/boxscores/199510290clt.htm
https://www.pro-football-reference.com/boxscores/199510290mia.htm
https://www.pro-football-reference.com/boxscores/199510290nwe.htm
https://www.pro-football-reference.com/boxscores/199510290phi.htm
https://www.pro-football-reference.com/boxscores/199510290pit.htm
https://www.pro-football-reference.com/boxscores/199510290sfo.htm
https://www.pro-football-reference.com/boxscores/199510290was.htm
https://www.pro-football-reference.com/boxscores/199510300min.htm
https://www.pro-football-reference.com/boxscores/199511050atl.htm
https://www.pro-football-reference.com/boxscores/199511050chi.htm
https://www.pro-football-reference.com/boxscores/199511050cin.htm
https://www.pro-football-reference.com/boxscores/199511050cle.htm
https://www.pro-football-reference.com/boxscores/199511050den.htm
https://www.pro-football-reference.com/boxscores/199511050clt.htm
https://www.pro-football-reference.com/boxscores/199511050kan.htm
https://ww

https://www.pro-football-reference.com/boxscores/199512250crd.htm
https://www.pro-football-reference.com/boxscores/199512300buf.htm
https://www.pro-football-reference.com/boxscores/199512300phi.htm
https://www.pro-football-reference.com/boxscores/199512310gnb.htm
https://www.pro-football-reference.com/boxscores/199512310sdg.htm
https://www.pro-football-reference.com/boxscores/199601060pit.htm
https://www.pro-football-reference.com/boxscores/199601060sfo.htm
https://www.pro-football-reference.com/boxscores/199601070dal.htm
https://www.pro-football-reference.com/boxscores/199601070kan.htm
https://www.pro-football-reference.com/boxscores/199601140dal.htm
https://www.pro-football-reference.com/boxscores/199601140pit.htm
https://www.pro-football-reference.com/boxscores/199601280dal.htm
https://www.pro-football-reference.com/boxscores/199609010rav.htm
https://www.pro-football-reference.com/boxscores/199609010car.htm
https://www.pro-football-reference.com/boxscores/199609010den.htm
https://ww

...

https://www.pro-football-reference.com/boxscores/200609100jax.htm
https://www.pro-football-reference.com/boxscores/200609100kan.htm
https://www.pro-football-reference.com/boxscores/200609100nwe.htm
https://www.pro-football-reference.com/boxscores/200609100nyg.htm
https://www.pro-football-reference.com/boxscores/200609100ram.htm
https://www.pro-football-reference.com/boxscores/200609100tam.htm
https://www.pro-football-reference.com/boxscores/200609100oti.htm
https://www.pro-football-reference.com/boxscores/200609110rai.htm
https://www.pro-football-reference.com/boxscores/200609110was.htm
https://www.pro-football-reference.com/boxscores/200609170atl.htm
https://www.pro-football-reference.com/boxscores/200609170rav.htm
https://www.pro-football-reference.com/boxscores/200609170chi.htm
https://www.pro-football-reference.com/boxscores/200609170cin.htm
https://www.pro-football-reference.com/boxscores/200609170dal.htm
https://www.pro-football-reference.com/boxscores/200609170den.htm
https://ww

https://www.pro-football-reference.com/boxscores/200611120clt.htm
https://www.pro-football-reference.com/boxscores/200611120jax.htm
https://www.pro-football-reference.com/boxscores/200611120mia.htm
https://www.pro-football-reference.com/boxscores/200611120min.htm
https://www.pro-football-reference.com/boxscores/200611120nwe.htm
https://www.pro-football-reference.com/boxscores/200611120nyg.htm
https://www.pro-football-reference.com/boxscores/200611120rai.htm
https://www.pro-football-reference.com/boxscores/200611120phi.htm
https://www.pro-football-reference.com/boxscores/200611120pit.htm
https://www.pro-football-reference.com/boxscores/200611120sea.htm
https://www.pro-football-reference.com/boxscores/200611120oti.htm
https://www.pro-football-reference.com/boxscores/200611130car.htm
https://www.pro-football-reference.com/boxscores/200611190crd.htm
https://www.pro-football-reference.com/boxscores/200611190rav.htm
https://www.pro-football-reference.com/boxscores/200611190car.htm
https://ww

https://www.pro-football-reference.com/boxscores/200701060sea.htm
https://www.pro-football-reference.com/boxscores/200701070nwe.htm
https://www.pro-football-reference.com/boxscores/200701070phi.htm
https://www.pro-football-reference.com/boxscores/200701130rav.htm
https://www.pro-football-reference.com/boxscores/200701130nor.htm
https://www.pro-football-reference.com/boxscores/200701140chi.htm
https://www.pro-football-reference.com/boxscores/200701140sdg.htm
https://www.pro-football-reference.com/boxscores/200701210chi.htm
https://www.pro-football-reference.com/boxscores/200701210clt.htm
https://www.pro-football-reference.com/boxscores/200702040clt.htm
Last site failed
https://www.pro-football-reference.com/boxscores/200709060clt.htm
https://www.pro-football-reference.com/boxscores/200709090buf.htm
https://www.pro-football-reference.com/boxscores/200709090cle.htm
https://www.pro-football-reference.com/boxscores/200709090dal.htm
https://www.pro-football-reference.com/boxscores/200709090g

https://www.pro-football-reference.com/boxscores/200710280oti.htm
https://www.pro-football-reference.com/boxscores/200710290den.htm
https://www.pro-football-reference.com/boxscores/200711040atl.htm
https://www.pro-football-reference.com/boxscores/200711040buf.htm
https://www.pro-football-reference.com/boxscores/200711040cle.htm
https://www.pro-football-reference.com/boxscores/200711040det.htm
https://www.pro-football-reference.com/boxscores/200711040clt.htm
https://www.pro-football-reference.com/boxscores/200711040kan.htm
https://www.pro-football-reference.com/boxscores/200711040min.htm
https://www.pro-football-reference.com/boxscores/200711040nor.htm
https://www.pro-football-reference.com/boxscores/200711040nyj.htm
https://www.pro-football-reference.com/boxscores/200711040rai.htm
https://www.pro-football-reference.com/boxscores/200711040phi.htm
https://www.pro-football-reference.com/boxscores/200711040tam.htm
https://www.pro-football-reference.com/boxscores/200711040oti.htm
https://ww

https://www.pro-football-reference.com/boxscores/200712240sdg.htm
https://www.pro-football-reference.com/boxscores/200712290nyg.htm
https://www.pro-football-reference.com/boxscores/200712300crd.htm
https://www.pro-football-reference.com/boxscores/200712300atl.htm
https://www.pro-football-reference.com/boxscores/200712300rav.htm
https://www.pro-football-reference.com/boxscores/200712300chi.htm
https://www.pro-football-reference.com/boxscores/200712300cle.htm
https://www.pro-football-reference.com/boxscores/200712300den.htm
https://www.pro-football-reference.com/boxscores/200712300gnb.htm
https://www.pro-football-reference.com/boxscores/200712300htx.htm
https://www.pro-football-reference.com/boxscores/200712300clt.htm
https://www.pro-football-reference.com/boxscores/200712300mia.htm
https://www.pro-football-reference.com/boxscores/200712300nyj.htm
https://www.pro-football-reference.com/boxscores/200712300rai.htm
https://www.pro-football-reference.com/boxscores/200712300phi.htm
https://ww

In [39]:
offdf.to_csv('...\\offense3.csv')